In [1]:
from pyexpat import model
from gensim.models import Word2Vec, KeyedVectors,FastText
import numpy as np
import os
current_path = ''#os.getcwd()

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 
from aksharamukha import transliterate
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
lang_mapping = {'hi':'Hindi',
                'bn':'Bengali',
                'ta':'Tamil',
                'gu':'Gujarati',
                'te':'Telugu',
                'kn':'Kannada',
                'ml':'Malayalam',
                'mr':'Marathi',
                'pa':'Punjabi (Gurmukhi)'}

In [21]:
lang_lst = ['bn','ta','gu','te','kn','ml','mr']

In [4]:
dim_lst = ['50','100']
emb_lst = ['fasttext','cbow','sg']
pca = PCA(n_components=2)
models = {}

In [14]:
def get_embb(loc,emb,lang,dim):
    words_vec = []
    dirpath=current_path+lang+"/"+dim+"/"+emb+"/"+lang+"-d"+dim+"-m2-"+emb+".model"
    if emb == 'fasttext':
        model = FastText.load(dirpath) #load the model
    else:
        model = Word2Vec.load(dirpath) #load the model
    filepath = current_path+loc+lang+".txt"
    with open(filepath,'r',encoding='utf-8') as file:
        found = []
        for lines in file:
            words=lines.strip()
            try:
                if lang == 'hi' or emb == 'fasttext':
                    sg_emb1 = model.wv[words]
                else:
                    new_word1= transliterate.process(lang_mapping[lang],
                                                     'Devanagari',
                                                     words,
                                                     nativize = False)
                    sg_emb1 = model.wv[new_word1]
                words_vec.append(sg_emb1)
                found.append(1)
            except:
                words_vec.append([-0.1]*int(dim))
                found.append(0)
                #print("words not found are : ",words)
    print("{}:found {} out of {} in {} {}".format(lang,sum(found),len(found),emb,dim))
    return np.array(words_vec),np.array(found)

In [31]:
def plott(eng_map,i,count,pca_result,lng_lst):
    for lang_index in range(len(lng_lst)):
        plt.scatter([pca_result[(count*lang_index)+i,0]],
                    [pca_result[(count*lang_index)+i,1]],
                    label=lang_mapping[lng_lst[lang_index]])
    plt.legend()
    plt.xlim([-0.1, 1.1])
    plt.ylim([-0.1, 1.1])
    plt.title(eng_words[i])
    plt.savefig(current_path+'plots/'+dim+'/'+emb+'/'+eng_map[eng_words[i]]+'_'+eng_words[i]+'.jpg')
    #plt.show()
    plt.close()

In [6]:
def get_filter(found):
    filter = []
    for i in range(len(found)):
        if found[i]:
            filter.append(i)
    return filter

In [32]:
def plot_all(eng_map,eng_words,emb,dim,found,emb_dct):
    count = int(sum(found))
    filter = get_filter(found)
    eng_words = eng_words[filter]
    lng_lst = list(emb_dct.keys())
    new_lst = []
    for lang in lng_lst:
        new_lst.extend(emb_dct[lang][filter].tolist())
    new_lst = np.array(new_lst)
    pca_result = pca.fit_transform(new_lst)
    pca_result = MinMaxScaler().fit_transform(pca_result)
    for i in range(count):
        plott(eng_map,i,count,pca_result,lng_lst)

In [33]:
def get_eng_words():
    eng_word = []
    filepath = current_path+"test/en.txt"
    with open(filepath,'r',encoding='utf-8') as file:
        for lines in file:
            eng_word.append(lines.strip())
    return np.array(eng_word)

In [34]:
def get_eng_map():
    eng_map = {}
    filepath = current_path+"en_map.txt"
    with open(filepath,'r',encoding='utf-8') as file:
        for lines in file:
            a = lines.strip().split(',')
            eng_map[a[0]] = a[1]
    return eng_map

In [76]:
def cos_sim(eng_map,eng_words,emb,dim,found,emb_dct):
    count = int(sum(found))
    filter = get_filter(found)
    eng_words = eng_words[filter]
    dct = {}
    langs = list(emb_dct.keys())
    for i in range(len(langs)-1):
        for j in range(i+1,len(langs)):
            key = langs[i]+'_'+langs[j]
            lang1 = emb_dct[langs[i]][filter]
            lang2 = emb_dct[langs[j]][filter]
            dct[key] = [[0,0] for _ in range(11)]
            for k in range(count):
                a = cosine_similarity(lang1[k:k+1],lang2[k:k+1])
                a = a.tolist()[0][0]
                dct[key][int(eng_map[eng_words[k]])][0] += a
                dct[key][int(eng_map[eng_words[k]])][1] += 1
            for k in range(11):
                dct[key][k] = dct[key][k][0] / dct[key][k][1]
    file = open(current_path+'cos_sim/'+dim+'/'+emb+'.csv','w')
    file.write('langs,fruits,plants,food,vegetables,colors,animals,possession,family,occupation,places,household-items\n')
    for key,value in dct.items():
        file.write(key+',')
        file.write(','.join(map(str,value)))
        file.write('\n')
    file.close()

# main code

In [22]:
for emb in emb_lst[1:]:
    for dim in dim_lst:
        hi_emb,_ = get_embb('train/',emb,'hi',dim)
        for lang in lang_lst:
            temp_emb,found  = get_embb('train/',emb,lang,dim)
            filter = get_filter(found)
            models[lang+'_'+dim+'_'+emb] = LinearRegression().fit(hi_emb[filter],temp_emb[filter])

hi:found 50 out of 50 in cbow 50
bn:found 48 out of 50 in cbow 50
ta:found 48 out of 50 in cbow 50
gu:found 49 out of 50 in cbow 50
te:found 50 out of 50 in cbow 50
kn:found 47 out of 50 in cbow 50
ml:found 49 out of 50 in cbow 50


/home/gautamc/Downloads/project/venv/lib/python3.8/site-packages/aksharamukha/transliterate.py:337: UserWarning: Source script: Marathi not found in the list of scripts supported. The text will not be transliterated.
  warnings.warn(script_not_found)


mr:found 49 out of 50 in cbow 50
hi:found 50 out of 50 in cbow 100
bn:found 48 out of 50 in cbow 100
ta:found 48 out of 50 in cbow 100
gu:found 49 out of 50 in cbow 100
te:found 50 out of 50 in cbow 100
kn:found 47 out of 50 in cbow 100
ml:found 49 out of 50 in cbow 100


/home/gautamc/Downloads/project/venv/lib/python3.8/site-packages/aksharamukha/transliterate.py:337: UserWarning: Source script: Marathi not found in the list of scripts supported. The text will not be transliterated.
  warnings.warn(script_not_found)


mr:found 49 out of 50 in cbow 100
hi:found 50 out of 50 in sg 50
bn:found 48 out of 50 in sg 50
ta:found 48 out of 50 in sg 50
gu:found 49 out of 50 in sg 50
te:found 50 out of 50 in sg 50
kn:found 47 out of 50 in sg 50
ml:found 49 out of 50 in sg 50


/home/gautamc/Downloads/project/venv/lib/python3.8/site-packages/aksharamukha/transliterate.py:337: UserWarning: Source script: Marathi not found in the list of scripts supported. The text will not be transliterated.
  warnings.warn(script_not_found)


mr:found 49 out of 50 in sg 50
hi:found 50 out of 50 in sg 100
bn:found 48 out of 50 in sg 100
ta:found 48 out of 50 in sg 100
gu:found 49 out of 50 in sg 100
te:found 50 out of 50 in sg 100
kn:found 47 out of 50 in sg 100
ml:found 49 out of 50 in sg 100


/home/gautamc/Downloads/project/venv/lib/python3.8/site-packages/aksharamukha/transliterate.py:337: UserWarning: Source script: Marathi not found in the list of scripts supported. The text will not be transliterated.
  warnings.warn(script_not_found)


mr:found 49 out of 50 in sg 100


In [23]:
eng_words = get_eng_words()

In [29]:
eng_map = get_eng_map()

In [77]:
for emb in emb_lst:
    for dim in dim_lst:
        emb_dct = {}
        emb_dct['hi'],found = get_embb('test/',emb,'hi',dim)
        for lang in lang_lst:
            temp_emb,new_found = get_embb('test/',emb,lang,dim)
            emb_dct[lang] = models[lang+'_'+dim+'_'+emb].predict(temp_emb)
            found = found & new_found
        
        cos_sim(eng_map,eng_words,emb,dim,found,emb_dct)
        plot_all(eng_map,eng_words,emb,dim,found,emb_dct)

hi:found 60 out of 60 in fasttext 50
bn:found 60 out of 60 in fasttext 50
ta:found 60 out of 60 in fasttext 50
gu:found 60 out of 60 in fasttext 50
te:found 60 out of 60 in fasttext 50
kn:found 60 out of 60 in fasttext 50
ml:found 60 out of 60 in fasttext 50
mr:found 60 out of 60 in fasttext 50
hi:found 60 out of 60 in fasttext 100
bn:found 60 out of 60 in fasttext 100
ta:found 60 out of 60 in fasttext 100
gu:found 60 out of 60 in fasttext 100
te:found 60 out of 60 in fasttext 100
kn:found 60 out of 60 in fasttext 100
ml:found 60 out of 60 in fasttext 100
mr:found 60 out of 60 in fasttext 100
hi:found 58 out of 60 in cbow 50
bn:found 54 out of 60 in cbow 50
ta:found 57 out of 60 in cbow 50
gu:found 58 out of 60 in cbow 50
te:found 55 out of 60 in cbow 50
kn:found 46 out of 60 in cbow 50
ml:found 56 out of 60 in cbow 50


/home/gautamc/Downloads/project/venv/lib/python3.8/site-packages/aksharamukha/transliterate.py:337: UserWarning: Source script: Marathi not found in the list of scripts supported. The text will not be transliterated.
  warnings.warn(script_not_found)


mr:found 59 out of 60 in cbow 50
hi:found 58 out of 60 in cbow 100
bn:found 54 out of 60 in cbow 100
ta:found 57 out of 60 in cbow 100
gu:found 58 out of 60 in cbow 100
te:found 55 out of 60 in cbow 100
kn:found 46 out of 60 in cbow 100
ml:found 56 out of 60 in cbow 100


/home/gautamc/Downloads/project/venv/lib/python3.8/site-packages/aksharamukha/transliterate.py:337: UserWarning: Source script: Marathi not found in the list of scripts supported. The text will not be transliterated.
  warnings.warn(script_not_found)


mr:found 59 out of 60 in cbow 100
hi:found 58 out of 60 in sg 50
bn:found 54 out of 60 in sg 50
ta:found 57 out of 60 in sg 50
gu:found 58 out of 60 in sg 50
te:found 55 out of 60 in sg 50
kn:found 46 out of 60 in sg 50
ml:found 56 out of 60 in sg 50


/home/gautamc/Downloads/project/venv/lib/python3.8/site-packages/aksharamukha/transliterate.py:337: UserWarning: Source script: Marathi not found in the list of scripts supported. The text will not be transliterated.
  warnings.warn(script_not_found)


mr:found 59 out of 60 in sg 50
hi:found 58 out of 60 in sg 100
bn:found 54 out of 60 in sg 100
ta:found 57 out of 60 in sg 100
gu:found 58 out of 60 in sg 100
te:found 55 out of 60 in sg 100
kn:found 46 out of 60 in sg 100
ml:found 56 out of 60 in sg 100


/home/gautamc/Downloads/project/venv/lib/python3.8/site-packages/aksharamukha/transliterate.py:337: UserWarning: Source script: Marathi not found in the list of scripts supported. The text will not be transliterated.
  warnings.warn(script_not_found)


mr:found 59 out of 60 in sg 100
